In [ ]:
from copy import *
from random import *
from jogos import *
from utils import *
import string
import copy
import jogar

from collections import namedtuple

stateBreakthru = namedtuple('EstadoBreakthru', 'to_move')

class EstadoBreakthru(stateBreakthru):
    
    def verificar_terminou(self):
        # 1 = White,  2 = Black
        if self.to_move == 2:
            for row in range(len(self.board)):
                if (self.board[0][row] == 'W'):
                    return 2 #Mudei por causa da func_aval
        elif self.to_move == 1:
            for row in range(len(self.board)):
                if (self.board[7][row] == 'B'):
                    #print("B in finishing pos")
                    return 1 #Mudei por causa da func_aval
        return 0
    
    def pos_convert(self, i, j):
        alfabeto = "abcdefgh"
        return alfabeto[j] + str(8 - i)

    def move_convert(self, move):
        numbers = []
        alfabeto = "abcdefgh"
        numbers.append(int(alfabeto.rfind(move[0]))) 
        numbers.append(int(move[1]) - 1)
        return numbers

    def __init__(self, to_move=1, n=8):
        self.board = [['.' for i in range(n)] for i in range(n)]
        #self.terminou = self.verificar_terminou()
        for i in range(2):
            for j in range(8):
                self.board[i][j] = 'B'

        for i in range(6, 8):
            for j in range(8):
                self.board[i][j] = 'W'


    def moves(self, to_move):
        list_moves = set()
        for i in range(0,8):
            for j in range(0,8):
                if self.board[i][j] == 'B' and to_move == 2 and j+1 <= 7:
                    if self.board[i+1][j] == '.' and i+1 <= 7:
                       list_moves.add("{pos1}-{pos2}".format(pos1=self.pos_convert(i,j), pos2=self.pos_convert(i+1,j)))
                    if (self.board[i+1][j+1] == 'W' or self.board[i+1][j+1] == '.') and i+1 <= 7 and j+1 <= 7:
                       list_moves.add("{pos1}-{pos2}".format(pos1=self.pos_convert(i,j), pos2=self.pos_convert(i+1,j+1)))
                    if (self.board[i+1][j-1] == 'W' or self.board[i+1][j-1] == '.') and i+1 <=7 and j-1 >= 0:
                        list_moves.add("{pos1}-{pos2}".format(pos1=self.pos_convert(i,j), pos2=self.pos_convert(i+1,j-1)))
                if self.board[i][j] == 'B' and to_move == 2 and j+1 == 8:
                    if self.board[i+1][j] == '.' and i+1 <= 7:
                        list_moves.add("{pos1}-{pos2}".format(pos1=self.pos_convert(i,j), pos2=self.pos_convert(i+1,j)))
                    if (self.board[i+1][j-1] == 'W' or self.board[i+1][j-1] == '.') and i+1 <= 7:
                        list_moves.add("{pos1}-{pos2}".format(pos1=self.pos_convert(i,j), pos2=self.pos_convert(i+1,j-1)))           

                if self.board[i][j] == 'W' and to_move == 1 and j+1 <= 7:
                    if self.board[i-1][j] == '.' and i-1 >= 0:
                        list_moves.add("{pos1}-{pos2}".format(pos1=self.pos_convert(i,j), pos2=self.pos_convert(i-1,j)))
                    if (self.board[i-1][j+1] == 'B' or self.board[i-1][j+1] == '.') and i-1 >= 0 and j+1 <= 7:
                        list_moves.add("{pos1}-{pos2}".format(pos1=self.pos_convert(i,j), pos2=self.pos_convert(i-1,j+1)))
                    if (self.board[i-1][j-1] == 'B' or self.board[i-1][j-1] == '.') and i-1 >= 0 and j-1 >= 0:
                        list_moves.add("{pos1}-{pos2}".format(pos1=self.pos_convert(i,j), pos2=self.pos_convert(i-1,j-1)))
                if self.board[i][j] == 'W' and to_move == 1 and j+1 == 8:
                    if self.board[i-1][j] == '.' and i-1 >= 0:
                        list_moves.add("{pos1}-{pos2}".format(pos1=self.pos_convert(i,j), pos2=self.pos_convert(i-1,j)))
                    if (self.board[i-1][j-1] == 'B' or self.board[i-1][j-1] == '.') and i-1 >= 0:
                        list_moves.add("{pos1}-{pos2}".format(pos1=self.pos_convert(i,j), pos2=self.pos_convert(i-1,j-1)))
        return sorted(list_moves)

## CLASS JOGO ##
class JogoBT_16(Game):
    """Play Breakthrough on an 8 x 8 board, with Max (first player) playing 'W' (1).
    """
    def __init__(self):
        self.initial = EstadoBreakthru(to_move=1)
        self.board = [['.' for i in range(8)] for i in range(8)]
        for i in range(2):
            for j in range(8):
                self.board[i][j] = 'B'

        for i in range(6, 8):
            for j in range(8):
                self.board[i][j] = 'W'

    def actions(self, state):
        "Legal moves for B and W"
        #Apenas queremos ver as ações possíveis para o jogador atual (to_move)
        return state.moves(state.to_move)

    def result(self, state, move):
        if state.to_move == 1:
            novo_estado = copy.copy(state)
            novo_estado = novo_estado._replace(to_move=2)
            novo_estado.board = state.board.copy()
            index_start = novo_estado.move_convert(move[0:2])
            new_index = novo_estado.move_convert(move[3:5])
            novo_estado.board[7 - index_start[1]][index_start[0]] = '.'
            novo_estado.board[7 - new_index[1]][new_index[0]] = 'W'
            return novo_estado
        else:
            novo_estado = copy.copy(state)
            novo_estado = novo_estado._replace(to_move=1)
            novo_estado.board = state.board.copy()
            index_start = novo_estado.move_convert(move[0:2])
            new_index = novo_estado.move_convert(move[3:5])
            novo_estado.board[7 - index_start[1]][index_start[0]] = '.'
            novo_estado.board[7 - new_index[1]][new_index[0]] = 'B'
            return novo_estado

    def terminal_test(self, state):
        "A state is terminal if the whites reached line 8 or some black reached line 1."
        return state.verificar_terminou() >= 1

    def display(self, state):
        symbols = "abcdefgh"
        print("-"*17)
        for i in range(len(state.board)):
            print("{num}|".format(num=8-i) + ' '.join(state.board[i]))
        print("-+" + "-"*15)
        print(" |" + ' '.join(i for i in symbols))

        if state.verificar_terminou() >= 1:
            return
        elif state.to_move == 1:
            print("--NEXT PLAYER: W")
        elif state.to_move == 2:
            print("--NEXT PLAYER: B")

    def executa(self, estado, listaJogadas):
        "executa varias jogadas sobre um estado dado"
        "devolve o estado final "
        s = estado
        for j in listaJogadas:
            s = self.result(s, j)
        return s
    
def func_aval_16(estado, jogador): #Acho que isto é a função heurística do nosso jogador o que faz sentido, é a função que ele utiliza para avaliar o move que vai efetuar? not sure tho
    terminou = estado.verificar_terminou()
    # O jogo ainda está a decorrer então vemos quem está na liderança(quem tiver a peça mais próxima). Em caso de empate
    # considera-se que quem jogou por último é que está a ganhar(ou seja, o jogador passado na func_aval no momento do empate)
    #if terminou == 0:
       #ficou em standby, mandei email ao professor a perguntar o que é que a func_aval é suposto fazer kkkkk
    pass

def func_aval_belarmino(estado, belarmino):
    pass



depth_for_all = 4 #o indicado pelo prof, não no enunciado, mas num email enviado a um aluno que depois postou no discord :)

belarmino = JogadorAlfaBeta("Belarmino",depth_for_all, func_aval_belarmino)


